In [1]:
import os
import nibabel as nib
import glob
from nilearn import image

In [2]:
# Specify input and output path
# For LINUX:
path ='ai4mi_project/data/segthor_train/train/'

# For WSL:
# path = '/home/{user}/ai4mi_project/data/segthor_train/train/'

In [3]:
# Get a list of all NIfTI files in the input directory

image_filenames = []

for i in range(1,41):
    
    image_filenames = image_filenames + sorted(glob.glob(path + 'Patient_'+ str(i).zfill(2) + '/*.nii.gz'))
    
# Remove extra GT file from Patient_27
image_filenames.remove(os.path.join(path, 'Patient_27/GT2.nii.gz'))

In [4]:
# Iter object for list names
it = iter(image_filenames)
# First new patient will have number 41, it will go up to 80
i = 41

for gt, pt in zip(it, it):
    
    patient_dir = os.path.join(path, 'Patient_'+ str(i).zfill(2))
    
    # Creating directory for new patient
    os.makedirs(patient_dir, exist_ok=True)
    
    gt_path = os.path.join(patient_dir, 'GT.nii.gz')
    pt_path = os.path.join(patient_dir, 'Patient_' + str(i).zfill(2) + '.nii.gz')
    
    if os.path.exists(gt_path) and os.path.exists(pt_path):
        print(f"Files for Patient_{str(i).zfill(2)} already exist. Skipping...")
        i += 1
        continue 
    
    # Loading files from original patient
    gt_file = nib.load(gt)
    pt_file = nib.load(pt)
    
    # NiLearn has its own function for applying Gaussian spatial smoothing to images as well. 
    # The only real difference from scipy.ndimage’s gaussian_filter() function is that instead of 
    # specifying the smoothing kernel in standard deviations, we specify it in units of 
    # full width half-maximum (FWHM). This is the standard way that most neuroimaging analysis packages 
    # specify smoothing kernel size, so it is preferable to SciPy’s approach. As the term implies, 
    # FWHM is the width of the smoothing kernel, in millimetres, at the point in the kernel where it is half 
    # of its maximum height. Thus a larger FWHM value applies more smoothing.
    fwhm = 4
    
    # Smoothing the patients' scan
    pt_smooth = image.smooth_img(pt_file, fwhm)
    
    # Saving the files for the new patient
    nib.save(gt_file, gt_path)
    nib.save(pt_smooth, pt_path)
    
    # Increasing patient number for the next patient.
    i += 1